# Crop raster based on bounding box

In [21]:
import rasterio
from rasterio.windows import from_bounds
from rasterio.coords import BoundingBox

# Define file paths
target_raster_path = "/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/HydroSHED/eu_acc_3s/eu_acc_3s.tif"
output_cropped_raster_path = "/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/processed_domain/MW_dem_3s.tif"

# Manually define the bounding box for cropping
manual_bounds = BoundingBox(left=38.34209196167711, 
                            bottom=18.779159074385802, 
                            right=43.337925295010244, 
                            top=23.74582574105227)

# Crop the target raster using the manual bounding box and save to a new file
with rasterio.open(target_raster_path) as target_raster:
    # Define a window for cropping based on the manually defined bounds
    window = from_bounds(manual_bounds.left, manual_bounds.bottom, manual_bounds.right, manual_bounds.top, target_raster.transform)
    
    # Update metadata for the cropped image
    cropped_meta = target_raster.meta.copy()
    cropped_meta.update({
        "height": window.height,
        "width": window.width,
        "transform": target_raster.window_transform(window)
    })

    # Read and write the cropped window directly to the output file
    with rasterio.open(output_cropped_raster_path, "w", **cropped_meta) as dest:
        for i in range(1, target_raster.count + 1):  # loop over bands
            data = target_raster.read(i, window=window)
            dest.write(data, i)

# Get stream network by applying are threshold to FAC.

In [17]:
import numpy as np
import rasterio

def calculate_river_network(fac_raster_path, area_matrix_km2, area_threshold_km2=6):
    """
    Generate a river network from a Flow Accumulation (FAC) raster, based on an area threshold.

    Parameters:
    fac_raster_path (str): Path to the Flow Accumulation (FAC) raster file.
    area_matrix_km2 (2D array): Pixel area matrix in square kilometers.
    area_threshold_km2 (float): Basin area size threshold to form a river network (default 6 km²).

    Returns:
    river_network (2D boolean array): Binary array where True indicates pixels part of the river network.
    """
    with rasterio.open(fac_raster_path) as src:
        fac_data = src.read(1)  # Read the first band (FAC data)

    # Calculate the accumulated area for each pixel
    accumulated_area_km2 = fac_data * area_matrix_km2

    # Generate the river network: True where accumulated area exceeds threshold
    river_network = accumulated_area_km2 >= area_threshold_km2

    return river_network

def save_river_network(river_network, reference_raster_path, output_path):
    """
    Save the river network as a new raster file.

    Parameters:
    river_network (2D boolean array): Binary array representing the river network.
    reference_raster_path (str): Path to a reference raster for metadata.
    output_path (str): Path to save the river network raster.
    """
    with rasterio.open(reference_raster_path) as src:
        meta = src.meta.copy()
        meta.update({
            "dtype": "uint8",
            "nodata": 0,
            "count": 1
        })

    # Convert boolean array to uint8 (1 for river pixels, 0 for non-river)
    river_network_uint8 = river_network.astype("uint8")

    # Save the river network to a new file
    with rasterio.open(output_path, "w", **meta) as dest:
        dest.write(river_network_uint8, 1)

def get_area_matrix_km2(raster_path):
    """
    Calculate the pixel area in square kilometers for a raster.

    Parameters:
    raster_path (str): Path to the input raster file.

    Returns:
    area_matrix_km2 (2D array): Matrix of pixel areas in square kilometers.
    """
    EARTH_RADIUS_M = 6378137  # Earth radius in meters

    # Open the raster and extract metadata
    with rasterio.open(raster_path) as src:
        transform = src.transform
        crs = src.crs
        height, width = src.height, src.width
        bounds = src.bounds

    # Initialize the area matrix
    area_matrix_km2 = np.zeros((height, width))

    # Check if the CRS is geographic (degrees)
    if crs.is_geographic:
        # Latitude for each row (pixel width varies with latitude)
        latitudes = np.linspace(bounds.top, bounds.bottom, height)
        
        # Pixel height in degrees (North-South)
        pixel_height_deg = abs(transform[4])
        pixel_height_m = pixel_height_deg * (np.pi / 180) * EARTH_RADIUS_M  # Convert degrees to meters

        for i, lat in enumerate(latitudes):
            # Pixel width in degrees (East-West), adjusted by latitude
            pixel_width_deg = abs(transform[0])
            pixel_width_m = pixel_width_deg * (np.pi / 180) * EARTH_RADIUS_M * np.cos(np.radians(lat))
            
            # Calculate pixel area in square meters and convert to square kilometers
            area_matrix_m2 = pixel_width_m * pixel_height_m
            area_matrix_km2[i, :] = area_matrix_m2 * 1e-6  # Convert to km²
    else:
        # CRS is projected, so pixel dimensions are already in meters
        pixel_width = abs(transform[0])
        pixel_height = abs(transform[4])
        
        # Calculate constant pixel area in square kilometers
        area_matrix_km2.fill((pixel_width * pixel_height) * 1e-6)  # Convert to km²

    return area_matrix_km2

# Paths and parameters
fac_raster_path = "/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/processed_domain/MW_acc_3s.tif"
reference_raster_path = fac_raster_path  # Using the FAC raster as a reference for output metadata
output_river_network_path = "/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/processed_domain/MW_3s_stream_1km2.tif"
area_threshold_km2 = 1

# Get the area matrix
area_matrix_km2 = get_area_matrix_km2(reference_raster_path)

# Generate the river network
river_network = calculate_river_network(fac_raster_path, area_matrix_km2, area_threshold_km2)

# Save the river network raster
save_river_network(river_network, reference_raster_path, output_river_network_path)

# Generate refMask based on DEM (reference raster).

In [22]:

import rasterio
import numpy as np

# Path to the reference file
fac_path = '/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/processed_domain/MW_dem_15s.tif'
# Save the binary mask to a new file
refMask_path = '/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/processed_domain/MW_refMask_15s.tif'

# Read the FAC file using a masked array
with rasterio.open(fac_path) as src:
    fac = src.read(1)  # Read the first band as a masked array
    meta = src.meta
    no_data = meta['nodata']
    meta.update(dtype=rasterio.float32, nodata=-9999)
    fac = fac.astype(np.float32)
    # Create a new raster with the same shape as dem_basic
    refMask = np.zeros_like(fac)
    # Assign values from 1 to n to non-NaN pixels
    mask = np.where(fac != no_data, True, False)
    n = np.count_nonzero(mask==True)
    refMask[mask] = range(1, n + 1)
    refMask[~mask] = meta['nodata']
    #refMask = np.where(refMask >= 1, refMask, meta['nodata'])

with rasterio.open(refMask_path, 'w', **meta) as dst:
    dst.write(refMask, 1)


# Download and preprocess land cover data

MODIS-based land cover (MCD12Q1) source: https://lpdaac.usgs.gov/products/mcd12q1v061/

Manully download land cover data in HDF format and store in the hdf_folder, run the following cell to convert hdf to tif.

The resulting tif would be processed later to match up with the reference raster.

In [23]:
from osgeo import gdal
import os

hdf_folder = '/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/LandCover/MCD12Q1v061'
output_folder = '/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/LandCover/MCD12Q1v061'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get a list of all HDF files in the folder
hdf_files = [file for file in os.listdir(hdf_folder) if file.endswith('.hdf')]

# Loop through each HDF file and save band 0 as GeoTIFF
for hdf_file in hdf_files:
    hdf_path = os.path.join(hdf_folder, hdf_file)
    hdf_ds = gdal.Open(hdf_path, gdal.GA_ReadOnly)
    band_ds = gdal.Open(hdf_ds.GetSubDatasets()[0][0], gdal.GA_ReadOnly)  # Open band 0
    band_array = band_ds.ReadAsArray()
    output_path = os.path.join(output_folder, f"{os.path.splitext(hdf_file)[0]}.tif")
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_path, band_ds.RasterXSize, band_ds.RasterYSize, 1, gdal.GDT_Int16, options=['COMPRESS=LZW', 'TILED=YES'])
    out_ds.SetGeoTransform(band_ds.GetGeoTransform())
    out_ds.SetProjection(band_ds.GetProjection())
    out_ds.GetRasterBand(1).WriteArray(band_array)
    out_ds.GetRasterBand(1).SetNoDataValue(-32768)

    # Close datasets
    out_ds = None
    band_ds = None
    hdf_ds = None


## Save the merged LCC to a folder that needs to be reprojected to match the reference raster with Soil properties.

In [25]:
import os
import glob
import rasterio
from rasterio.merge import merge
from rasterio.enums import Resampling

input_folder = '/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/LandCover/MCD12Q1v061'
merged_LCC_tiff = '/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/processed_domain/param_raw/LCC.tif'
os.makedirs(os.path.dirname(merged_LCC_tiff), exist_ok=True)

# Find all GeoTIFF files in the input folder
tif_files = glob.glob(os.path.join(input_folder, '*.tif'))

# Open each GeoTIFF file
src_files_to_mosaic = []
for tif_file in tif_files:
    src = rasterio.open(tif_file)
    src_files_to_mosaic.append(src)

# Merge all opened GeoTIFF files
mosaic, out_trans = merge(src_files_to_mosaic, resampling=Resampling.nearest)

# Create output GeoTIFF file
profile = src_files_to_mosaic[0].profile
profile.update({
    'height': mosaic.shape[1],
    'width': mosaic.shape[2],
    'transform': out_trans
})

# Write the merged GeoTIFF to the desired location
with rasterio.open(merged_LCC_tiff, 'w', **profile) as dst:
    dst.write(mosaic)

# Close all opened GeoTIFF files
for src in src_files_to_mosaic:
    src.close()


## Compute IM.tif from the merged LCC.tif

In [2]:
import os
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np

# Define file paths
impervious_raster_path = "/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/processed_domain/param_raw/LCC.tif"
basin_raster_path = "/Users/qyang/Downloads/data/CREST/New_domains/Whole_NCM_domain/phase2_domain/Whole_domain/Whole_Saudi/basic/dem.tif"
output_fraction_path = "/Volumes/TiPlus7100Mac/D_drive/NCM_data/Hydro-fabric/Topography/processed_domain/param/IM_Saudi.tif"
os.makedirs(os.path.dirname(output_fraction_path), exist_ok=True)

# Open the impervious raster
with rasterio.open(impervious_raster_path) as impervious_raster:
    # Create a binary mask: 1 for impervious (value 13), 0 otherwise
    impervious_data = impervious_raster.read(1)
    binary_mask = (impervious_data == 13).astype(np.float32)

    # Define the metadata for the binary mask for reprojection
    mask_meta = impervious_raster.meta.copy()
    mask_meta.update(dtype=rasterio.float32)

# Open the basin raster as the reference for projection and resolution
with rasterio.open(basin_raster_path) as basin_raster:
    basin_meta = basin_raster.meta.copy()
    basin_shape = basin_raster.shape
    basin_transform = basin_raster.transform

    # Prepare an empty array to hold the reprojected impervious fraction
    impervious_fraction = np.empty(basin_shape, dtype=np.float32)

    # Reproject and resample the binary mask to match the basin raster
    reproject(
        source=binary_mask,
        destination=impervious_fraction,
        src_transform=mask_meta['transform'],
        src_crs=mask_meta['crs'],
        dst_transform=basin_transform,
        dst_crs=basin_raster.crs,
        resampling=Resampling.average  # Use average resampling to get fraction
    )

# Save the impervious fraction raster
basin_meta.update(dtype=rasterio.float32, compress='lzw')

with rasterio.open(output_fraction_path, 'w', **basin_meta) as output_raster:
    output_raster.write(impervious_fraction, 1)